<div align="center">
    <a href="https://colab.research.google.com/github/z-a-f/PyTorch-Notebooks/blob/master/01-Basic-Neural-Networks/Binary-Classification.ipynb" style="text-align:center">Open in Colab
        <img src="https://colab.research.google.com/img/colab_favicon.ico" width=64/>
    </a>
</div>

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

# Binary Classification: Classifying Movie Reviews

In this section you will cover binary classification, which is one of the most widely applied kind of ML problem. For that purpose you will use IMDB dataset <sup>$\dagger$</sup>, which is a collection of moview reviews, with the reviews being either positive or negative. The classification task is to predict the sentiment of the review.

---
<small><sup>$\dagger$</sup>Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). **Learning Word Vectors for Sentiment Analysis**. *The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011)*.</small>

## IMDB Dataset

IMDB dataset is a set of 50,000 polarized movie reviews. The dataset is split into two equal parts of 25,000 reviews, one for training and another for testing. At the same time the every subset has equal number of positive and negative reviews.

The reason for separating the dataset into two parts (train and test), is to avoid “memorization”. Imagine that your network is very powerful, and after seeing 50,000 reviews, it was able to memorize all of them. You will get 100% accuracy for any example in that dataset. However, when presented with the examples never seen before, such a network will fail. Separating the data into train and test sets allows us to train on one part of the data, and check the accuracy on the other.

To load the dataset, you will use the TorchText. Note that the dataset will download ~80 MB of data, and exapand into ~0.5GB.